In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
import numpy as np
import category_encoders as ce
import warnings
from pycaret.classification import *
from imblearn.over_sampling import SMOTE
warnings.filterwarnings('ignore')


In [ ]:
df_train = pd.read_csv("../input/hackerearth-customer-segmentation-hackathon/avhacklive/Train.csv")
df_test = pd.read_csv("../input/hackerearth-customer-segmentation-hackathon/avhacklive/Test.csv")
df_test_id = df_test["id"]

In [ ]:
df_train

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.dtypes

In [ ]:
df_train["days_since_prev_campaign_contact"].value_counts()

In [ ]:
df_train["customer_age"].value_counts().sort_index()

In [ ]:
df_train["marital"] = df_train["marital"].fillna("unknown")
df_test["marital"] = df_test["marital"].fillna("unknown")

In [ ]:
ma = df_train[df_train["marital"]=="married"]["customer_age"].mean()
uma = df_train[df_train["marital"]!="married"]["customer_age"].mean()
uk = df_train[df_train["marital"]=="unknown"]["customer_age"].mean()

In [ ]:
ma

In [ ]:
df_train["marital"].value_counts()

In [ ]:
for i in range(len(df_train)):
    if np.isnan(df_train["customer_age"][i]):
        # print(df_train["customer_age"][i])
        if df_train["marital"][i] == "single":
            df_train["customer_age"][i] = round(uma)
        if df_train["marital"][i] == "married":
            df_train["customer_age"][i] = round(ma)
        if df_train["marital"][i] == "divorced":
            df_train["customer_age"][i] = round(ma)
        if df_train["marital"][i] == "unknown":
            df_train["customer_age"][i] = round(uk)

for i in range(len(df_test)):
    if np.isnan(df_test["customer_age"][i]):
        if df_test["marital"][i] == "single":
            df_test["customer_age"][i] = round(uma)
        if df_test["marital"][i] == "married":
            df_test["customer_age"][i] = round(ma)
        if df_test["marital"][i] == "divorced":
            df_test["customer_age"][i] = round(ma)
        if df_test["marital"][i] == "unknown":
            df_test["customer_age"][i] = round(uk)



In [ ]:
df_train["job_type"].value_counts()

In [ ]:
def one_hot_encoding(df,col):
    one_hot_encoder=ce.OneHotEncoder(cols=col,return_df=True,use_cat_names=True)
    df_final = one_hot_encoder.fit_transform(df)
    return df_final

In [ ]:
df_train["term_deposit_subscribed"].value_counts()

In [ ]:
df_train

In [ ]:
for i in df_train.columns:
    if df_train[i].dtypes == "object":
        print(i)
        print(len(df_train[i].value_counts()))

In [ ]:
df_train

In [ ]:
for i in df_train.columns:
    if i == "term_deposit_subscribed":
        continue
    if df_train[i].dtypes == "object" and i !="id":
        # print(i)
        df_train[i].fillna("unknown",inplace=True)
        df_train = one_hot_encoding(df_train,i)
        # print(len(df_train[i].value_counts()))
    elif df_train[i].isnull().sum()>0: 
        if i == "num_contacts_in_campaign":
            df_train[i].fillna(1.0,inplace=True)
        else:
            df_train[i].fillna(round(df_train[i].mean()),inplace=True)
    
    if df_test[i].dtypes == "object" and i !="id":
        # print(i)
        df_test[i].fillna("unknown",inplace=True)
        df_test = one_hot_encoding(df_test,i)
        # print(len(df_test[i].value_counts()))
    elif df_test[i].isnull().sum()>0: 
        if i == "num_contacts_in_campaign":
            df_test[i].fillna(1.0,inplace=True)
        else:
            df_test[i].fillna(round(df_test[i].mean()),inplace=True)

In [ ]:
columns_to_drop = ["id"]
df_train = df_train.drop(columns_to_drop,axis = 1)
df_test = df_test.drop(columns_to_drop,axis = 1)

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.dtypes

In [ ]:
X_train = df_train.drop("term_deposit_subscribed",axis = 1)
Y_train = df_train["term_deposit_subscribed"]

In [ ]:
smote = SMOTE()
X_train, Y_train = smote.fit_resample(X_train, Y_train)

In [ ]:
Y_train.value_counts()

In [ ]:
df_train_smoted = X_train
df_train_smoted["term_deposit_subscribed"] = Y_train

In [ ]:
experiment = setup(df_train_smoted,target = "term_deposit_subscribed",imputation_type="iterative")

In [ ]:
# compare_models()

In [ ]:
mod = create_model("rf")

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
y_pred = predict_model(mod , data = df_test)

In [ ]:
final = pd.DataFrame()
final["id"] = df_test_id
final["term_deposit_subscribed"]=round(y_pred["Score"])
final.to_csv("submission_v1.csv")